## Snohomish County Ballot Drop Box Collection Routing


This code was developed to take in a list of ballot boxes and workers and generate optimal routes for collection. Best use case is during Presidential Elections when all 35 boxes are open and routes can be set up in a variety of ways. Once the routes are created, there will be an option to push them to a feature layer that is connected to the Snohomish County Ballot Collection Dashboard for viewing there. There is also an option to assign routes to workers through batch assignment to Workforce. If created routes are not functional/preferred then please manually create all assignments in Workforce. This code is primarily built for large scale routing and bulk assigning. Any additions/changes to Assignments should be done by Dispatchers in the Workforce app.

There will be notes ahead of each chunk of text with instructions for editable fields - keep an eye out for changes required to boxes, workers, and parameters for routing. Note that routing costs 1 credit per route created every time the cell is ran. Be careful to not unecessarily run the cell as to not waste credits.

For non-coding experts: putting a hashtag (#) in front of any line of code comments it out and the line is not ran as actual code. This routing tool will require some commenting and uncommenting of ballot boxes and workers to make modifications prior to routing.

#### Make sure to run ALL cells that are needed for setup, cells that don't NEED to be run for further segments to funtion will be clearly marked as OPTIONAL.

#### 1. Run this cell to connect to your GIS and get started:

In [112]:
import arcgis
from arcgis.gis import GIS
gis = GIS("home")

You are logged on as lander03_UW with an administrator role, proceed with caution.


In [113]:
import pandas as pd
from arcgis.apps import workforce
from arcgis.apps.workforce import Project
from arcgis.geocoding import geocode
pd.options.display.max_columns = None
import random
from datetime import datetime, timedelta, timezone
from arcgis.apps.workforce import Assignment
from datetime import datetime
from arcgis.features import Feature, FeatureSet
print("Import Successful")

Import Successful


#### 2. Importing the map of all ballot boxes in Snohomish County and setting the start location to Everett Campus.
If there are any changes to ballot boxes update this Feature Layer and or/ update the ID to the correspond to the correct Feature Layer.

In [110]:
# MODIFY the feature layer id as necessary
fl_id = "6816bfbff1354d67b0a8f333e1577e5a" # Map Name: ballotboxlocationOD
#================================================================================================ Do not modify beyond this point

# Fetching the Feature Service - verifies that the search for the map name returns results and prints detials about the map found
map_search = gis.content.search(f'id:"{fl_id}"')
if map_search:
    map_item = map_search[0]
    print(f"Successfully loaded Feature Layer:\n"
          f"   • Title: {map_item.title}\n"
          f"   • ID: {map_item.id}\n"
          f"   • Owner: {map_item.owner}")
    # Creating the data frame with all boxes in Snohomish County
    ballotbox_layer = map_item.layers[0]
    ballotbox_df = ballotbox_layer.query(where="1=1",out_fields="OBJECTID,Drop_Box_location,Destination,City,State,Origin", return_geometry=True, as_df=True)
    print(f"\nSuccessfully Loaded Ballot Box Locations: {len(ballotbox_df)} Boxes in Snohomish County")
else:
    print("No Feature Layer found with that title.")

# Sets starting location to Everett Campus
start_address = "3000 Rockefeller Avenue, Everett, WA"
start_location_name = "Everett Campus"
location = geocode(start_address, out_sr={"wkid": 102100})[0]["location"]
location["spatialReference"] = {"wkid": 102100}

# Creates a Feature Layer for the start location that will be using in routing
feature = arcgis.features.Feature(
    geometry=location,
    attributes={"ObjectID": 1, "Name": "Everett Campus"}
)
feature_set = arcgis.features.FeatureSet([feature])
feature_collection = arcgis.features.FeatureCollection.from_featureset(feature_set)
start_layer = {
    "layerDefinition": feature_collection.properties["layers"][0]["layerDefinition"],
    "featureSet": feature_set.value
}

print(f"Successfully loaded Start Location to {start_location_name}")

Successfully loaded Feature Layer:
   • Title: ballotboxlocationOD
   • ID: d80141d96f7e4bf882b4434592283e75
   • Owner: vtan114_UW

Successfully Loaded Ballot Box Locations: 35 Boxes in Snohomish County
Successfully loaded Start Location to Everett Campus


#### OPTIONAL
Run this cell to visualize the original ballot box data frame. Uncomment the line to run if desired, shouldn't be necessary unless box names in the next chunk need to be double checked.

In [1]:
#print(ballotbox_df)

#### 3. Modifying the list of boxes to include all boxes the SHOULD BE INCLUDED IN ROUTING.
##### Comment (put a hashtag in front) of boxes that should not be included. The exact location names from Drop_box_location column in the original ballotbox_df are required, avoid deleting or modifying this list to maintain the correct references. Run the above cell to see the options for boxes.
The number of ballot boxes selected will be printed at the bottom cell for verification that the correct number of boxes were selected.

In [75]:
# EDIT HERE - place a hashtag in front of any boxes that should not be routed
included_boxes = [
    "Arlington",
    "Bothell",
    "Brier",
    "Darrington",
    "Edmonds",
    "Everett (College)",
    "Everett (Courthouse)",
    "Everett (Mall)",
    "Everett (McCollum Park)",
    "Goldbar",
    "Granite Falls",
    "Index",
    "Lake Stevens",
    "Lakewood",
    "Lynnwood (Ash Way)",
    "Lynnwood (City Hall)",
    "Lynnwood (Edmonds College)",
    "Marysville (Ash Ave Park & Ride)",
    "Marysville (Grove Elementary)",
    "Marysville (State Ave)",
    "Mill Creek",
    "Monroe",
    "Mountlake Terrace",
    "Mukilteo",
    "Silvana",
    "Smokey Point",
    "Snohomish (Centennial Middle School)",
    "Snohomish (GPHS)",
    "Snohomish (Library)",
    "Stanwood",
    "Startup",
    "Sultan",
    "Tulalip",
    "Woodinville",
    "Woodway"
]
#================================================================================================ Do not modify beyond this point

ballotbox_df = ballotbox_df[ballotbox_df["Drop_Box_location"].isin(included_boxes)]
print(f"Successfully Modified Ballot Box Selection: {len(ballotbox_df)} Boxes Selected")

Successfully Modified Ballot Box Selection: 35 Boxes Selected


#### 4. Import the Workforce Project
If there is a new project created, make sure to link it to this Notebook by modifying the project_id

In [76]:
# MODIFY the project ID as necessary
project_id = "157bf167353e4db3a9e3f739937d0535" # Project Name: Snohomish County Ballot Collection Workforce (Feature Service)
#================================================================================================ Do not modify beyond this point

# Searching for and saving the Workforce project
project_item = gis.content.get(project_id)
if project_item:
    assignments_layer = project_item.layers[0]
    workers_layer = project_item.layers[1]
    workforce_project = Project(project_item)
    print(f"Successfully loaded Workforce Project:\n"
          f"   • Title: {workforce_project.title}\n"
          f"   • ID: {workforce_project.id}\n"
          f"   • Owner: {workforce_project.owner}")
else:
    workforce_project = None
    print("No Workforce project found with that ID")

Successfully loaded Workforce Project:
   • Title: Snohomish County Ballot Collection Workforce
   • ID: a717bb96df4b45b69549b53834c5e188
   • Owner: <User username:lander03_UW>


#### 5. Clearing out Yesterday's Workforce Assignments

In [77]:
all_assignments = workforce_project.assignments.search()
today = datetime.now(timezone.utc).date()
assignments_to_delete = [
    assignment for assignment in all_assignments
    if assignment.assigned_date and assignment.assigned_date.date() < today
]
print(f"Found {len(assignments_to_delete)} assignments older than today.")
if assignments_to_delete:
    workforce_project.assignments.batch_delete(assignments_to_delete)
    print("Old assignments successfully deleted.")
else:
    print("No old assignments found.")

Found 0 assignments older than today.
No old assignments found.


#### 6. Create the Routes!
### WARNING!
#### This chunk uses 1 credit per route to run, update parameters carefully and avoid running multiple times. Note that modifications to Assignments can be done manually in Workforce.
##### Edit the parameters before running. This may take a few minutes to run

The more strict the parameters are, the harder it will be to generate usable routes so be careful with this. The routing logic will aim to create the minimum number of routes while fitting in the other parameters, if adjusting the number of routes consider lowering the max_stops and max_route_time to be stricter and force the routing to balance work among the teams more efficiently.

Note: 
Most Optimal Parameters for all 35 Boxes and 5 Routes: max_stops = 8, stop_service_time = 15, max_route_time = 300

In [56]:
# EDIT these parameters to designate routing restrictions
num_routes = 5  # Number of Routes that will be created
max_stops = 8  # Maximum number of stops per route
now = datetime.now()
departure_time = now.replace(hour=7, minute=0, second=0, microsecond=0)
# If it's already past 7:00 AM, set to 7:00 AM tomorrow
if now >= departure_time:
    departure_time += timedelta(days=1)
stop_service_time = 15  # Service time per stop (minutes)
max_route_time = 300  # Maximum route duration (minutes)
return_to_start = True  # Designates that all routes should end at Everett Campus
#================================================================================================ Do not modify beyond this point

# Running the plan routes tool
results = arcgis.features.analysis.plan_routes(
                                    ballotbox_df,          # stops_layer
                                    num_routes,               # route_count
                                    max_stops,                # max_stops_per_route
                                    departure_time,           # route_start_time
                                    start_layer,              # start_layer
                                    stop_service_time=stop_service_time,
                                    max_route_time=max_route_time,
                                    travel_mode="driving time",
                                    return_to_start=return_to_start
                                    )
print("Routes successfully created")

# Querying the results to use later
routes = results['routes_layer'].query().sdf
stops = results['assigned_stops_layer'].query().sdf
stops["Drop_Box_location"] = stops["Drop_Box_location"].fillna("Everett Campus")
unassigned_stops_layer = results.get('unassigned_stops_layer')
# Adding a RouteID Field that we can reference later
routes['RouteNumber'] = routes['RouteName'].str.extract(r'Route\s*?(\d+)', expand=False)
routes['RouteID'] = "Route " + routes['RouteNumber']
stops['RouteNumber'] = stops['RouteName'].str.extract(r'Route\s*?(\d+)', expand=False)
stops['RouteID'] = "Route " + stops['RouteNumber']

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Through Traffic Prohibited").
{"cost": 5.0}


Routes successfully created


#### 7. Print Route Summary
##### If the routes that were created don't make sense or aren't work exporting, stop here and manually assign routes in Workforce.

In [114]:
unassigned_df = None

# Handle unassigned stops
if unassigned_stops_layer:
    unassigned_df = unassigned_stops_layer.query().sdf.copy()
    unassigned_df['RouteID'] = 'Unassigned'
    unassigned_df['Sequence'] = None
    unassigned_df['EstimatedReturnTime'] = None
    print("Unassigned Stops:")
    print(unassigned_df[['Drop_Box_location', 'Street', 'City']])
else:
    print("All selected drop boxes were assigned to a route.\n")

print("-" * 40)

# Create route summary
summary = routes.groupby('RouteID').agg({
    'StopCount': 'sum',
    'TotalTime': 'sum',
    'TotalTravelTime': 'sum',
    'TotalStopServiceTime': 'sum',
    'Total_Miles': 'sum',
    'Total_Kilometers': 'sum'
}).reset_index().round(2)

# Filter and merge assigned stops
assigned_stops = stops[~stops["StopType"].isin(["Route start", "Route end"])].copy()
assigned_stops.sort_values(by=["RouteID", "Sequence"], inplace=True)
merged = assigned_stops.merge(summary, on="RouteID", how="left")

# Add estimated return time
merged['EstimatedReturnTime'] = merged['TotalTime'].apply(
    lambda mins: (departure_time + timedelta(minutes=mins)).strftime("%I:%M %p")
)

# Common export columns
export_columns = [
    'RouteID', 'Sequence', 'Drop_Box_location', 'City',
    'StopCount', 'TotalTime', 'TotalTravelTime',
    'TotalStopServiceTime', 'Total_Miles', 'Total_Kilometers',
    'EstimatedReturnTime'
]

# Print route summaries and stops
for route_id, group in merged.groupby("RouteID"):
    row = group.iloc[0]
    total_minutes = row['TotalTime']
    travel_minutes = row['TotalTravelTime']

    total_time_str = f"{int(total_minutes // 60)} hr {int(total_minutes % 60)} min"
    travel_time_str = f"{int(travel_minutes // 60)} hr {int(travel_minutes % 60)} min"
    return_time = row['EstimatedReturnTime']

    print(f"{route_id}")
    print(f"{row['StopCount']} Stops | Travel Time: {travel_time_str} | "
          f"Total Time: {total_time_str} | Distance: {row['Total_Miles']} miles | "
          f"Est. Return Time: {return_time}")

    for i, (_, stop) in enumerate(group.iterrows(), start=1):
        print(f"  {i}. {stop['Drop_Box_location']}")

    print("-" * 40)

# Format assigned stops for export
assigned_export = merged[export_columns]

# Format unassigned stops for export
if unassigned_df is not None:
    unassigned_export = unassigned_df.loc[:, ['RouteID', 'Sequence', 'Drop_Box_location', 'City']].copy()
    for col in export_columns[5:]: 
        unassigned_export.loc[:, col] = None
    export_df = pd.concat([assigned_export, unassigned_export], ignore_index=True)
else:
    export_df = assigned_export

All selected drop boxes were assigned to a route.

----------------------------------------
Route 1
8 Stops | Travel Time: 1 hr 37 min | Total Time: 3 hr 37 min | Distance: 43.16 miles | Est. Return Time: 10:37 AM
  1. Everett (Courthouse)
  2. Snohomish (Library)
  3. Snohomish (GPHS)
  4. Everett (McCollum Park)
  5. Mill Creek
  6. Lynnwood (Ash Way)
  7. Mukilteo
  8. Everett (Mall)
----------------------------------------
Route 2
8 Stops | Travel Time: 1 hr 49 min | Total Time: 3 hr 49 min | Distance: 54.47 miles | Est. Return Time: 10:49 AM
  1. Bothell
  2. Woodinville
  3. Brier
  4. Mountlake Terrace
  5. Woodway
  6. Edmonds
  7. Lynnwood (Edmonds College)
  8. Lynnwood (City Hall)
----------------------------------------
Route 3
6 Stops | Travel Time: 2 hr 46 min | Total Time: 4 hr 16 min | Distance: 118.01 miles | Est. Return Time: 11:16 AM
  1. Smokey Point
  2. Arlington
  3. Darrington
  4. Silvana
  5. Stanwood
  6. Lakewood
----------------------------------------
Rout

#### If the routes created are not functional, either re-run after adjusting parameters OR move to manual assignment in Workforce. 
##### If manually assigning in Workforce, run the first two chunks in Step 10: Send the Routes to Ballot Collection Dashboard to connect and clear the old routes from the Dashboard and Workforce maps.

#### 8. Import the Workers: designate which workers will be working today (assigned routes).
##### Modify the list of workers by commenting out names of workers not working today. Make sure to account for ALL routes
Add names as needed ensuring that the names added have already been added as workers to the Workforce project, use their name as it appears in Workforce. Note that the only workers in the list should be those that will be assigned routes through this code. Assignments can be created manually anytime in Workforce to workers not in this list.

In [97]:
# EDIT the route-to-worker assignment
route_to_worker_name = {
    "Route 1": "UW_Washington" # This is your only worker in Workforce right now, so I put it in here to test :

    # EDIT HERE: Modify in this way when more workers are added to Workforce!
    #"Route 2": "Laura Marie",
    #"Route 3": "Gabe", 
    #"Route 4": "Josiah", 
    #"Route 5": "Gabe" --> note that we can assign multiple routes to one worker
}
#================================================================================================ Do not modify beyond this point

# Extracting unique worker names from the route mapping and searching Workforce for available workers
all_workers = set(route_to_worker_name.values())
imported_workers = workforce_project.workers.search()
working_today = [w for w in imported_workers if w.name in all_workers]
worker_names_today = [w.name for w in working_today]
worker_count = len(working_today)
all_worker_names = [worker.name for worker in imported_workers]
print("All Workers in Workforce:")
print(", ".join(all_worker_names))
print(f"\nWorkers to be Routed Today: {worker_count} Workers")
print(", ".join(worker_names_today))

All Workers in Workforce:
Gabe, Victoria, Laura Marie, Bella, Josiah

Workers to be Routed Today: 1 Workers
Gabe


##### After verifying that all Workers are found and in the list of workers to be routed today, run this cell to map the routes to workers.

In [98]:
workers_raw = workers_layer.query(
    where="1=1",
    out_fields="GlobalID, OBJECTID, name, status",
    return_geometry=False
)

name_to_globalid = {
    f.attributes["name"]: f.attributes["GlobalID"].upper()
    for f in workers_raw.features
    if f.attributes["name"] in all_workers
}

active_workers = list(name_to_globalid.values())
if active_workers:

    # === STEP 5: Map each route to a worker GlobalID ===
    # Also validates all names exist
    route_to_worker = {
        route: name_to_globalid[name]
        for route, name in route_to_worker_name.items()
        if name in name_to_globalid
    }
    
    missing_names = [name for name in route_to_worker_name.values() if name not in name_to_globalid]
    if missing_names:
        raise ValueError(f"The following worker names are missing in 'workers_layer': {missing_names}")
    
    print("Route to Worker Mapping (Name + GlobalID):")
    for route, name in route_to_worker_name.items():
        global_id = name_to_globalid.get(name, "Not Found")
        print(f"{route} -> {name} ({global_id})")
else:
    print("No active workers, please list workers to assign routes to.")

Route to Worker Mapping (Name + GlobalID):
Route 1 -> Gabe (1FD88FD4-7C07-4E99-B846-23422D10A36C)
Route 2 -> Gabe (1FD88FD4-7C07-4E99-B846-23422D10A36C)
Route 3 -> Gabe (1FD88FD4-7C07-4E99-B846-23422D10A36C)
Route 4 -> Gabe (1FD88FD4-7C07-4E99-B846-23422D10A36C)
Route 5 -> Gabe (1FD88FD4-7C07-4E99-B846-23422D10A36C)


#### 9. Export Assignments to Workforce
##### Takes the workers that we imported previosly and batch sends the assignments to Workforce. Adjustments to the assignment, priority, worker assigned, etc. can all be done in the Workforce App.

In [99]:
assignments_to_add = []
for i, row in routes.iterrows():
    route_name = row["RouteName"]
    route_ID = row["RouteID"]

    # Get worker GlobalID from your mapping
    worker_globalid = route_to_worker.get(route_ID)
    if worker_globalid is None:
        print(f"No worker assigned to {route_ID}. Skipping.")
        continue

    # Find matching Worker object
    worker = next(
        (w for w in imported_workers if w.global_id.upper() == worker_globalid),
        None
    )
    if worker is None:
        print(f"Worker with GlobalID {worker_globalid} not found in Workforce project. Skipping {route_ID}.")
        continue

    # Get route stops (skip Everett Campus)
    route_stops = stops[
        (stops["RouteName"] == route_name) &
        (stops["Drop_Box_location"] != "Everett Campus")
    ].sort_values("Sequence")

    # Create an assignment for each stop
    for _, stop in route_stops.iterrows():
        sequence = stop["Sequence"]
        location_name = stop["Drop_Box_location"]
        geometry = stop["SHAPE"]

        assignment = Assignment(
            project=workforce_project,
            assignment_type="Drop Box Ballot Collection",  # <-- Make sure this matches exactly to Workforce
            location=f"Stop {sequence - 1}: {location_name} ({route_ID})",
            description=f"{route_ID}",
            status="assigned",
            worker=worker,
            assigned_date=datetime.now(),
            due_date=stop["DepartTimeUTC"],
            geometry=geometry
        )
        assignments_to_add.append(assignment)

# Batch add all assignments
try:
    assignments = workforce_project.assignments.batch_add(assignments_to_add)
    print(f"{len(assignments)} assignments successfully sent to Workforce.")
except Exception as e:
    print("Error sending assignments:", e)

35 assignments successfully sent to Workforce.


#### 10. Send the Routes to Ballot Collection Dashboard

##### Connecting to the Routes and Stops Feature Layer

In [61]:
# These layers are added to the Snohomish County Ballot Collection Dispatcher Map for visibility on the dashboard - you can hide them at any time
routes_feature_layer_id = "cc05a86529604439b9bf47bf1ad58f69" # Layer Name: Routes_Layer
stops_feature_layer_id = "1ede5afe0360411a8299a92c869a6040" # Layer Name: Stops_Layer
#================================================================================================ Do not modify beyond this point

# Accessing the feature layers
stops_fl_item = gis.content.get(stops_feature_layer_id)
stops_layer_featurelayer = stops_fl_item.layers[0]
routes_fl_item = gis.content.get(routes_feature_layer_id)
routes_layer_featurelayer = routes_fl_item.layers[0]

print("Successfully loaded routes and stops feature layers")

Successfully loaded routes and stops feature layers


##### Delete Old Routes and Stops from Feature Layers

In [63]:
stops_layer_featurelayer.delete_features(where="1=1")
routes_layer_featurelayer.delete_features(where="1=1")
print("Successfully deleted old routes and stops from feature layers")

Successfully deleted old routes and stops from feature layers


##### Send new Routes and Stops to Feature Layers

In [66]:
# Saving the layers for modification
routes_fl_item = gis.content.get(routes_feature_layer_id)
routes_layer_featurelayer = routes_fl_item.layers[0]
stops_fl_item = gis.content.get(stops_feature_layer_id)
stops_layer_featurelayer = stops_fl_item.layers[0]

# Convert DataFrame rows into Feature objects
def df_to_features(df, geometry_col='SHAPE'):
    features = []
    for _, row in df.iterrows():
        # Ensuring attributes are plain Python types
        attributes = row.drop(geometry_col).to_dict()
        # Converting numpy types to Python types
        for key, value in attributes.items():
            if hasattr(value, 'item'):
                attributes[key] = value.item()
        # Cleaning geometry
        geometry = row[geometry_col]
        if hasattr(geometry, 'as_dict'):
            geometry = geometry.as_dict
        elif isinstance(geometry, dict):
            geometry = geometry
        else:
            raise ValueError(f"Unsupported geometry type: {type(geometry)}")
        features.append(Feature(geometry=geometry, attributes=attributes))
    return features

new_route_features = df_to_features(routes)
new_stop_features = df_to_features(stops)

# Build OBJECTID -> RouteID lookups
route_id_lookup = dict(zip(routes['OBJECTID'], routes['RouteID']))
stop_id_lookup = dict(zip(stops['OBJECTID'], stops['RouteID']))

# Overwrite RouteName on each route and stop feature using RouteID
for feat in new_route_features:
    oid = feat.attributes.get("OBJECTID")
    if oid in route_id_lookup:
        feat.attributes["RouteName"] = route_id_lookup[oid]
for feat in new_stop_features:
    oid = feat.attributes.get("OBJECTID")
    if oid in stop_id_lookup:
        feat.attributes["RouteName"] = stop_id_lookup[oid]

# Update ROUTES layer
routes_layer_featurelayer.delete_features(where="1=1")
routes_layer_featurelayer.edit_features(adds=new_route_features)
print("Routes successfully added to existing feature layer.")

# Update STOPS layer
stops_layer_featurelayer.delete_features(where="1=1")
stops_layer_featurelayer.edit_features(adds=new_stop_features)
print("Stops successfully added to the existing feature layer.")

Routes successfully added to existing feature layer.
Stops successfully added to the existing feature layer.


#### 11. Formatting the Layers for Visibility

In [104]:
# Predefined colors for first 10 routes
colors = {
    f"Route {i}": color for i, color in enumerate([
        [140, 115, 173, 255],  # Husky Purple (Light)
        [235, 120, 125, 255],  # Cardinal Red (Light)
        [102, 180, 132, 255],  # Evergreen (Light)
        [151, 214, 240, 255],  # Sky Blue (Light)
        [70, 130, 180, 255],     # Steel Blue,  # Midnight Blue (Light)
        [235, 165, 175, 255],  # Dusty Rose (Light)
        [200, 190, 220, 255],  # Soft Lavender (Light)
        [200, 200, 200, 255],  # Warm Gray (Light)
        [255, 210, 102, 255],  # Sunshine Gold (Light)
        [100, 170, 175, 255],  # Deep Teal (Light)
    ], start=1)
}
# Random color for the remaining routes (if there are any)
def random_color():
    return [random.randint(0, 255) for _ in range(3)] + [255]

# Query features to get route names
features = routes_layer_featurelayer.query(where="1=1", out_fields="RouteName", return_geometry=False).features
route_names = sorted(set(f.attributes['RouteName'] for f in features))

# Unique value renderer
unique_values = []
for route_name in route_names:
    color = colors.get(route_name, random_color())
    unique_values.append({
        "value": route_name,
        "symbol": {
            "type": "esriSLS",
            "style": "esriSLSSolid",
            "color": color,
            "width": 3
        },
        "label": route_name
    })

# Define and apply renderer
renderer = {
    "type": "uniqueValue",
    "field1": "RouteName",
    "uniqueValueInfos": unique_values,
    "defaultSymbol": {
        "type": "esriSLS",
        "style": "esriSLSSolid",
        "color": [100, 100, 100, 255],
        "width": 2
    },
    "defaultLabel": "Other"
}
routes_layer_featurelayer.manager.update_definition({"drawingInfo": {"renderer": renderer}})

# Query features to get route names
features = stops_layer_featurelayer.query(where="1=1", out_fields="RouteName", return_geometry=False).features
route_names = sorted(set(f.attributes['RouteName'] for f in features if f.attributes['RouteName']))

# Unique value renderer
unique_values = []
for route_name in route_names:
    color = colors.get(route_name, random_color())
    unique_values.append({
        "value": route_name,
        "symbol": {
            "type": "esriSMS",  # Simple Marker Symbol
            "style": "esriSMSCircle",
            "color": color,
            "size": 8,
            "outline": {
                "color": [0, 0, 0, 255],
                "width": 1
            }
        },
        "label": route_name
    })
    
# Define and apply renderer
renderer = {
    "type": "uniqueValue",
    "field1": "RouteName",
    "uniqueValueInfos": unique_values,
    "defaultSymbol": {
        "type": "esriSMS",
        "style": "esriSMSCircle",
        "color": [150, 150, 150, 255],
        "size": 6,
        "outline": {
            "color": [0, 0, 0, 255],
            "width": 1
        }
    },
    "defaultLabel": "Other"
}
stops_layer_featurelayer.manager.update_definition({"drawingInfo": {"renderer": renderer}})

print("Route and Stops successfully recolored")

Route and Stops successfully recolored


#### 12. Exporting Route Results to CSV

In [105]:
# Adding the workers as a column for the csv export
assigned_export = assigned_export.copy()
assigned_export["AssignedWorker"] = assigned_export["RouteID"].map(route_to_worker_name)

# Export
today_str = datetime.today().strftime("%Y-%m-%d")
filename = f"routes_{today_str}.csv"
assigned_export.to_csv(filename, index=False)
print(f"CSV exported as {filename}")

CSV exported as routes_2025-06-11.csv


#### OPTIONAL: Delete all old Workforce data from the Map - this clears the casche so do this carefully!!

In [106]:
assignments_layer = workforce_project._item.layers[0]

# Query all features to get their IDs
results = assignments_layer.query(where="1=1", return_ids_only=True)
object_ids = results.get('objectIds', [])

# Finding and deleting assignments
if object_ids:
    print(f"Deleting all {len(object_ids)} assignments.")
    delete_result = assignments_layer.delete_features(deletes=','.join(map(str, object_ids)))
    print("Successfully deleted all results")
else:
    print("No assignments found to delete.")

Deleting all 234 assignments.
Successfully deleted all results
